In [3]:
import os
import pandas as pd

In [7]:
ls_df = []
for stats in os.listdir("stats"):
    df = pd.read_csv(f"stats/{stats}")

    # get name of the file
    name = stats.split(".")[0]
    benchmark, file_type = name.split("_")
    print(benchmark, file_type)

    # add columns
    df["benchmark"] = benchmark
    df["file_type"] = file_type

    # append to list
    ls_df.append(df)

benchmark1 csv
benchmark2 parquet
benchmark2 csv
benchmark1 parquet


In [8]:
# report
df_report = pd.concat(ls_df)
df_report.head()

,Run,Polars Time,Polars Memory (MB),DuckDB Time,DuckDB Memory (MB),benchmark,file_type
0,1,33.697651,24286.570312,42.122135,3148.527344,benchmark1,csv
1,2,53.738236,27145.773438,42.881872,3287.472656,benchmark1,csv
2,3,56.419289,27004.015625,42.387825,3122.972656,benchmark1,csv
3,4,56.272037,27013.207031,42.140444,3116.722656,benchmark1,csv
4,5,55.046149,27050.335938,42.698847,3107.722656,benchmark1,csv


In [22]:
# calculate statistics mean (+/- std) for benchmark and file_type
df_stats = df_report.groupby(["benchmark", "file_type"]).agg(["mean", "std"])
df_stats

Run            Polars Time            \
                      mean        std        mean       std   
benchmark  file_type                                          
benchmark1 csv         3.0   1.581139   51.034672  9.751892   
           parquet    25.5  14.577380    0.516810  0.031671   
benchmark2 csv         3.0   1.581139   50.424432  9.907560   
           parquet    10.5   5.916080    0.710596  0.033101   

                     Polars Memory (MB)              DuckDB Time            \
                                   mean          std        mean       std   
benchmark  file_type                                                         
benchmark1 csv             26499.980469  1238.605785   42.446225  0.337473   
           parquet          2649.062422    68.589109    0.172825  0.008948   
benchmark2 csv             25613.060156  1246.166134   42.493363  0.618833   
           parquet           594.104687    17.857971    0.847980  0.031329   

                     DuckDB Memory (MB)             
                                   mean        std  
benchmark  file_type                                
benchmark1 csv              3156.683594  74.670577  
           parquet          2710.440469   0.760861  
benchmark2 csv               963.829687  31.213209  
           parquet           164.952344   0.451078

In [30]:
# calculate statistics mean (+/- std) for benchmark and file_type
df_stats = df_report.groupby(["benchmark", "file_type"]).mean().round(2)

# replace benchmark1 = simple query, benchmark2 = complex query
df_stats = df_stats.reset_index()
df_stats["benchmark"] = df_stats["benchmark"].replace({"benchmark1": "simple query", "benchmark2": "complex query"})
df_stats

,benchmark,file_type,Run,Polars Time,Polars Memory (MB),DuckDB Time,DuckDB Memory (MB)
0,simple query,csv,3.0,51.03,26499.98,42.45,3156.68
1,simple query,parquet,25.5,0.52,2649.06,0.17,2710.44
2,complex query,csv,3.0,50.42,25613.06,42.49,963.83
3,complex query,parquet,10.5,0.71,594.10,0.85,164.95


In [35]:
df_time = df_stats[["benchmark", "file_type", "Polars Time", "DuckDB Time"]]
df_time

,benchmark,file_type,Polars Time,DuckDB Time
0,simple query,csv,51.03,42.45
1,simple query,parquet,0.52,0.17
2,complex query,csv,50.42,42.49
3,complex query,parquet,0.71,0.85
